In [3]:
## Librerías
import numpy as np
import seaborn as sns
import sys
import cv2 as cv2
import glob
from PIL import Image
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import gc
import keras
from sklearn.model_selection import train_test_split
import os

import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers
tfd = tfp.distributions

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import umap
from sklearn.preprocessing import StandardScaler

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import glob

import random 

In [4]:
## Uso de GPU
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

## DATASET DE MICROGRAFÍAS

In [5]:
### Rutas de todas las imágenes

## Obtener todas las rutas de las imágenes en el dataset
ruta_dataset  = glob.glob('/home/willy98/archivos/Datasets/SH_procesadas/Recortadas_grises/*/*/*/*.jpg')

## Capturar solo rutas secas 
rutas_imagenes_secas = []
rutas_imagenes_humedas = []

clases_secas= []
clases_humedas = []
for ruta in ruta_dataset:
    if ruta.split('/')[-2] == 'Secas' or ruta.split('/')[-2] == 'Seca':
        clases_secas.append(ruta.split('/')[-4])
        rutas_imagenes_secas.append(ruta)
    else:
        clases_humedas.append(ruta.split('/')[-4])
        rutas_imagenes_humedas.append(ruta)

## Construcción de datasets
data_secas = pd.DataFrame({'ruta':rutas_imagenes_secas,'clase':clases_secas})
data_humedas = pd.DataFrame({'ruta':rutas_imagenes_humedas,'clase':clases_humedas})

## reemplazo de etiquetas}
lables_generalization = {'CuNi1':'MF1','CuNi2':'MF2','CuNi3':'MF3'}
data_secas.replace(lables_generalization,inplace=True)
data_humedas.replace(lables_generalization,inplace=True)

## summary    
print('Imágenes secas:', np.shape(rutas_imagenes_secas)[0],' - Imágenes Húmedas: ',np.shape(rutas_imagenes_humedas)[0])
print('MF1:', '- secas: ', np.shape(data_secas[data_secas['clase']=='MF1'])[0], '- húmedas: ', np.shape(data_humedas[data_humedas['clase']=='MF1'])[0])
print('MF2:', '- secas: ', np.shape(data_secas[data_secas['clase']=='MF2'])[0], '- húmedas: ', np.shape(data_humedas[data_humedas['clase']=='MF2'])[0])
print('MF3:', '- secas: ', np.shape(data_secas[data_secas['clase']=='MF3'])[0], '- húmedas: ', np.shape(data_humedas[data_humedas['clase']=='MF3'])[0])

Imágenes secas: 931  - Imágenes Húmedas:  792
MF1: - secas:  379 - húmedas:  265
MF2: - secas:  289 - húmedas:  265
MF3: - secas:  263 - húmedas:  262


In [6]:
### Construir Datasets

# Imágenes secas
secas_arrays = []
secas_labels = []

## Dimensión de las imágenes
nx = 128
ny = 128

## recorrer rutas
for i in range(len(data_secas)):    
    ## Capturar imágenes 
    ruta = data_secas.iloc[i].ruta
    label = data_secas.iloc[i].clase
    
    ## Leer imágenes en 128x128
    img = tf.keras.preprocessing.image.load_img(ruta, target_size=(nx, ny),color_mode = "grayscale")
    
    ## Convertir a array
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    
    ## Normalizar
    img_array = img_array / 255.0
    
    ## Agregar a listas
    secas_arrays.append(img_array)
    secas_labels.append(label)
    
#imagenes húmedas
humedas_arrays = []
humedas_labels = []

## recorrer rutas
for i in range(len(data_humedas)):
     ## Capturar imágenes 
    ruta = data_secas.iloc[i].ruta
    label = data_secas.iloc[i].clase
    
    ## Leer imágenes en 128x128
    img = tf.keras.preprocessing.image.load_img(ruta, target_size=(nx, ny),color_mode = "grayscale")
    
    ## Convertir a array
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    
    ## Normalizar
    img_array = img_array / 255.0
    
    ## Agregar a listas
    humedas_arrays.append(img_array)
    humedas_labels.append(label) 

In [7]:
### Construcción del dataset de entrenamiento y de prueba
X_train_secas, X_test_secas, y_train_secas, y_test_secas = train_test_split(secas_arrays, secas_labels, test_size=0.20, random_state=42)

X_train_humedas, X_test_humedas, y_train_humedas, y_test_humedas = train_test_split(humedas_arrays, humedas_labels, test_size=0.20, random_state=42)

print("Secas: *training: ",np.shape(X_train_secas), "test: ", np.shape(X_test_secas))
print("Humedas: *training: ",np.shape(X_train_humedas), "test: ", np.shape(X_test_humedas))

Secas: *training:  (744, 128, 128, 1) test:  (187, 128, 128, 1)
Humedas: *training:  (633, 128, 128, 1) test:  (159, 128, 128, 1)


In [8]:
x_train_secas = np.array(X_train_secas)
x_test_secas = np.array(X_test_secas)

x_train_secas = x_train_secas.astype('float32')
x_test_secas= x_test_secas.astype('float32')
 
#y_test = tf.keras.utils.to_categorical(y_test_secas)
#y_train_secas = tf.keras.utils.to_categorical(y_train_secas) 

#x_train_secas /= 255
#x_test /= 255

print('x_train shape:', x_train_secas.shape)
print('Number of images in x_train', x_train_secas.shape[0], "y_train:", np.shape(y_train_secas))
print('Number of images in x_test', x_test_secas.shape, "y_test: ", np.shape(y_test_secas))

x_train shape: (744, 128, 128, 1)
Number of images in x_train 744 y_train: (744,)
Number of images in x_test (187, 128, 128, 1) y_test:  (187,)


In [9]:
print(type(x_train_secas),' : ',np.shape(x_train_secas))

<class 'numpy.ndarray'>  :  (744, 128, 128, 1)


## AUTOENCODER CONVENCIONAL (AE)

In [ ]:
latent_dim = 128
nfilts = 512

encoder_inputs = keras.Input(shape=(128, 128,1))

x = tf.keras.layers.Conv2D(nfilts, (3, 3), activation='relu', padding='same',name='layer_E1')(encoder_inputs)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/2, (3, 3), activation='relu', padding='same',name='layer_E3')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/4, (3, 3), activation='relu', padding='same',name='layer_E2')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)


x = tf.keras.layers.Conv2D(nfilts/8, (3, 3), activation='relu', padding='same',name='layer_E7')(x)
x = layers.Flatten()(x)

outputs = layers.Dense(latent_dim, activation="relu",name='encoded')(x)

# Reshape para reconstruir la última convolucional
ndim = 16
x = layers.Reshape((ndim,ndim*2,1))(outputs)

## Capas convolucionales
x = tf.keras.layers.Conv2D(nfilts, (3, 3), activation='relu', padding='same',name='layer_D1')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/2, (3, 3), activation='relu', padding='same',name='layer_D2')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/4, (3, 3), activation='relu', padding='same',name='layer_D4')(x)
x = tf.keras.layers.UpSampling2D((2, 1))(x)
x = tf.keras.layers.BatchNormalization()(x)


decoder_outputs = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same',name='decoded')(x)

autoencoder = tf.keras.models.Model(encoder_inputs, decoder_outputs)
autoencoder.summary()

In [ ]:
type(X_train_secas)

#### Entrenamiento (Secas)

In [ ]:
opt = tf.optimizers.Adam(learning_rate = 0.000001)
autoencoder.compile(optimizer=opt,loss="binary_crossentropy")
autoencoder.fit(X_train_secas, X_train_secas, epochs=2000,
                batch_size=32, shuffle=True,
                         callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=120)],
                         validation_data=(X_test_secas, X_test_secas))

In [ ]:
autoencoder.save('models/AE_dry_1_128.h5')

#### Entrenamiento (Húmedas)

In [ ]:
opt = tf.optimizers.Adam(learning_rate = 0.000001)
autoencoder.compile(optimizer=opt,loss="binary_crossentropy")
autoencoder.fit(X_train_secas, X_train_secas, epochs=2000,
                batch_size=32, shuffle=True,
                         callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=120)],
                         validation_data=(X_test_secas, X_test_secas))

In [ ]:
autoencoder.save('models/AE_wet_1_128.h5')

## AUTOENCODER VARIACIONAL GAUSSIANO (GVAE)

In [10]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [11]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")        

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
            
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction /= tf.reduce_max(reconstruction)
            
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            
            '''
            reconstruction_loss =tf.reduce_mean(
                tf.reduce_sum( tf.keras.losses.MeanSquaredError()(data, reconstruction)
                             )
            )
            '''
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))            
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.reconstruction = reconstruction
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)       
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),            
        }

#### Encoder

In [12]:
latent_dim = 128
nfilts = 512

encoder_inputs = keras.Input(shape=(nx, ny,1))

x = tf.keras.layers.Conv2D(nfilts, (3, 3), activation='relu', padding='same',name='layer_E1')(encoder_inputs)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/2, (3, 3), activation='relu', padding='same',name='layer_E3')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/4, (3, 3), activation='relu', padding='same',name='layer_E2')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/8, (3, 3), activation='relu', padding='same',name='layer_E7')(x)
x = layers.Flatten()(x)



z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(inputs=encoder_inputs, outputs=[z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               


2023-07-31 01:11:52.630651: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8584 MB memory:  -> device: 0, name: NVIDIA TITAN V, pci bus id: 0000:1b:00.0, compute capability: 7.0


                                )]                                                                
                                                                                                  
 layer_E1 (Conv2D)              (None, 128, 128, 51  5120        ['input_1[0][0]']                
                                2)                                                                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 64, 64, 512)  0           ['layer_E1[0][0]']               
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 512)  2048       ['max_pooling2d[0][0]']          
 alization)                                                                                       
                                                                                                  
 layer_E3 

#### Decoder

In [13]:
## Entrada Z
latent_inputs = keras.Input(shape=(latent_dim,))

##dimensiones
ndim = 8
nfilts = 256

## anti-flatten de la última capa convolucional
x = layers.Dense(ndim * ndim * nfilts, activation="relu")(latent_inputs)

## Reshape para reconstruir la última convolucional
x = layers.Reshape((ndim,ndim, nfilts))(x)

## Capas convolucionales
x = tf.keras.layers.Conv2D(nfilts, (3, 3), activation='relu', padding='same',name='layer_D1')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/2, (3, 3), activation='relu', padding='same',name='layer_D2')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/4, (3, 3), activation='relu', padding='same',name='layer_D4')(x)
x = tf.keras.layers.UpSampling2D((4, 4))(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/8, (3, 3), activation='relu', padding='same',name='layer_D6')(x)

decoder_outputs = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same',name='layer_D7')(x)

## Construcción del decoder
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense (Dense)               (None, 16384)             2113536   
                                                                 
 reshape (Reshape)           (None, 8, 8, 256)         0         
                                                                 
 layer_D1 (Conv2D)           (None, 8, 8, 256)         590080    
                                                                 
 up_sampling2d (UpSampling2D  (None, 16, 16, 256)      0         
 )                                                               
                                                                 
 batch_normalization_3 (Batc  (None, 16, 16, 256)      1024      
 hNormalization)                                           

#### Entrenamiento (Secas)

In [ ]:
vae = VAE(encoder, decoder)
opt = tf.optimizers.Adam(learning_rate = 0.000001)
#nll = lambda x , rv_x: -rv_x.log_prob(x)
vae.compile(optimizer=opt)


vae.fit(x_train_secas, epochs=2000, 
                batch_size=32,
                         callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30)])

Epoch 1/2000


2023-07-31 01:11:56.650054: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-31 01:11:58.622939: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f84b7ffef20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-31 01:11:58.622987: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA TITAN V, Compute Capability 7.0
2023-07-31 01:11:58.629654: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-31 01:11:58.805835: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


24/24 [==============================] - 13s 118ms/step - loss: 34534.2962 - reconstruction_loss: 34842.3789 - kl_loss: 0.0698
Epoch 2/2000
24/24 [==============================] - 2s 82ms/step - loss: 34456.6114 - reconstruction_loss: 34683.9258 - kl_loss: 0.0691
Epoch 3/2000
24/24 [==============================] - 2s 80ms/step - loss: 34698.7811 - reconstruction_loss: 34501.2070 - kl_loss: 0.0685
Epoch 4/2000
24/24 [==============================] - 2s 80ms/step - loss: 34035.6602 - reconstruction_loss: 34120.8008 - kl_loss: 0.0685
Epoch 5/2000
24/24 [==============================] - 2s 82ms/step - loss: 33891.2461 - reconstruction_loss: 33958.3945 - kl_loss: 0.0680
Epoch 6/2000
24/24 [==============================] - 2s 82ms/step - loss: 34040.1905 - reconstruction_loss: 33633.1562 - kl_loss: 0.0677
Epoch 7/2000
24/24 [==============================] - 2s 81ms/step - loss: 33023.4638 - reconstruction_loss: 33275.0820 - kl_loss: 0.0701
Epoch 8/2000
24/24 [=========================

24/24 [==============================] - 2s 84ms/step - loss: 19662.8168 - reconstruction_loss: 19399.8535 - kl_loss: 46.4528
Epoch 61/2000
24/24 [==============================] - 2s 83ms/step - loss: 19022.1030 - reconstruction_loss: 18773.6621 - kl_loss: 59.4265
Epoch 62/2000
24/24 [==============================] - 2s 83ms/step - loss: 17920.1518 - reconstruction_loss: 17701.4590 - kl_loss: 75.7878
Epoch 63/2000
24/24 [==============================] - 2s 82ms/step - loss: 17199.3921 - reconstruction_loss: 16846.0508 - kl_loss: 95.1347
Epoch 64/2000
24/24 [==============================] - 2s 81ms/step - loss: 16509.3927 - reconstruction_loss: 16240.3564 - kl_loss: 118.1768
Epoch 65/2000
24/24 [==============================] - 2s 82ms/step - loss: 15582.3377 - reconstruction_loss: 15416.1553 - kl_loss: 145.1729
Epoch 66/2000
24/24 [==============================] - 2s 82ms/step - loss: 15044.9579 - reconstruction_loss: 14695.6553 - kl_loss: 175.9499
Epoch 67/2000
24/24 [==========

24/24 [==============================] - 2s 83ms/step - loss: 11243.9282 - reconstruction_loss: 10814.0557 - kl_loss: 407.5281
Epoch 119/2000
24/24 [==============================] - 2s 83ms/step - loss: 11196.3018 - reconstruction_loss: 10801.7627 - kl_loss: 404.4466
Epoch 120/2000
24/24 [==============================] - 2s 83ms/step - loss: 11226.3661 - reconstruction_loss: 10838.4365 - kl_loss: 398.1804
Epoch 121/2000
24/24 [==============================] - 2s 82ms/step - loss: 11166.6479 - reconstruction_loss: 10809.5205 - kl_loss: 395.5148
Epoch 122/2000
24/24 [==============================] - 2s 83ms/step - loss: 11235.6950 - reconstruction_loss: 10816.3535 - kl_loss: 393.2694
Epoch 123/2000
24/24 [==============================] - 2s 82ms/step - loss: 11154.7629 - reconstruction_loss: 10794.8877 - kl_loss: 386.8766
Epoch 124/2000
24/24 [==============================] - 2s 82ms/step - loss: 11204.2972 - reconstruction_loss: 10808.9346 - kl_loss: 381.2271
Epoch 125/2000
24/24 

In [ ]:
### Guardar modelo
vae.encoder.save('models/GVAE_encoder_dry_1_128.h5')
vae.decoder.save('models/GVAE_decoder_dry_1_128.h5')

#### Entrenamiento (Húmedas)

In [ ]:
vae = VAE(encoder, decoder)
opt = tf.optimizers.Adam(learning_rate = 0.000001)
#nll = lambda x , rv_x: -rv_x.log_prob(x)
vae.compile(optimizer=opt)


vae.fit(X_train_secas, epochs=2000, 
                batch_size=32,
                         callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30)])

In [ ]:
### Guardar modelo
vae.encoder.save('models/GVAE_encoder_wet_1_128.h5')
vae.decoder.save('models/GVAE_decoder_wet_1_128.h5')